# Building an Agent around a Query Pipeline

## SETUP

In [5]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
from openai import OpenAI

llm = OpenAI(api_key=OPENAI_API_KEY)
completion = llm.chat.completions.create(
    model="gpt-4o", messages=[{"role": "user", "content": "write a haiku about ai"}]
)

print(completion.choices[0].message)

I0000 00:00:1727380311.795364 6661049 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


ChatCompletionMessage(content="Echoes in circuits,\nSilicon dreams intertwine,\nFuture's neural dance.", role='assistant', function_call=None, tool_calls=None, refusal=None)


- to use duckdb from sqlalchemy (duckdb_engine 0.13.2)[https://pypi.org/project/duckdb_engine/]

In [7]:
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine(
    "sqlite:///../data/Chinook.db",
    # "duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/netflix.db",
)

sql_database = SQLDatabase(engine=engine)

### Setup Observability

In [8]:
from llama_index.core.query_pipeline import QueryPipeline

In [9]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## SETUP Text2SQL query engine tool

In [11]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    verbose=True,
)

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=("Useful for translating a natural language query into a SQL query"),
)

## Setup ReAct Agent Pipeline


In [12]:
from llama_index.core.query_pipeline import QueryPipeline as QP

qp = QP(verbose=True)

### Define Agent Input Component


In [13]:
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep,
)
from llama_index.core.agent import Task, AgentChatResponse
from llama_index.core.query_pipeline import (
    StatefulFnComponent,
    QueryComponent,
    ToolRunnerComponent,
)
from llama_index.core.llms import MessageRole
from typing import Dict, Any, Optional, Tuple, List, cast


# Input Component
## This is the component that produces agent inputs to the rest of the components
## Can also put initialization logic here.
def agent_input_fn(state: Dict[str, Any]) -> str:
    """Agent input function.

    Returns:
        A Dictionary of output keys and values. If you are specifying
        src_key when defining links between this component and other
        components, make sure the src_key matches the specified output_key.

    """
    task = state["task"]
    if len(state["current_reasoning"]) == 0:
        reasoning_step = ObservationReasoningStep(observation=task.input)
        state["current_reasoning"].append(reasoning_step)
    return task.input


agent_input_component = StatefulFnComponent(fn=agent_input_fn)

ImportError: cannot import name 'StatefulFnComponent' from 'llama_index.core.query_pipeline' (/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/pract-llamaindex/.venv/lib/python3.10/site-packages/llama_index/core/query_pipeline/__init__.py)